In [2]:
!pip install statsmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 11.3 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [statsmodels] [statsmodels]


In [3]:
import pandas as pd
import statsmodels.api as sm
import joblib

# --- This script should be run separately to train and save the model ---

def train_and_save_volume_forecaster(data_path='sample-final-data.csv', model_path='volume_forecaster_model.joblib'):
    """
    Trains a SARIMA model to forecast hourly patient volume.
    """
    print("Training patient volume forecaster...")
    # Load and preprocess data
    df = pd.read_csv(data_path, encoding='utf-8-sig')
    df['Arrival TimeDT'] = pd.to_datetime(df['Arrival TimeDT'], format='%m/%d/%y %H:%M')
    
    # Aggregate patient counts by hour
    df.set_index('Arrival TimeDT', inplace=True)
    hourly_volume = df.resample('H').size().rename('patient_volume')
    
    # SARIMA model requires seasonality parameters (m)
    # m=24 for daily patterns, m=24*7 for weekly patterns. We'll use daily.
    sarima_model = sm.tsa.statespace.SARIMAX(
        hourly_volume,
        order=(1, 1, 1),              # (p, d, q) - Non-seasonal components
        seasonal_order=(1, 1, 1, 24), # (P, D, Q, m) - Seasonal components (daily)
        enforce_stationarity=False,
        enforce_invertibility=False
    ).fit()
    
    # Save the trained model
    joblib.dump(sarima_model, model_path)
    print(f"Volume forecasting model saved to {model_path}")

# Run the training function
# train_and_save_volume_forecaster()